In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from keras import applications
from keras.layers import GlobalAveragePooling2D,Dropout , Dense 
from keras.models import Model


Using TensorFlow backend.


In [2]:
CATEGORIES = []
imgSize = 64
Data =[]
DATADIR = "C:/Users/hala/Downloads/carsData/ford/fordNet"
listDir  = os.listdir(DATADIR)
for category in listDir:
    CATEGORIES.append(category)


In [3]:
def create_training_data():
    
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                image= cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                if((image is None)==False):
                    newImage = cv2.resize(image , (imgSize , imgSize))
                    Data.append([newImage , class_num])
            except Exception as e:
                pass
        


In [4]:
Data = []
trainingData = []
testData = []
create_training_data()
numClasse = 0
index1 = 0
index2=0
while(index2<len(Data)):
    while((index2<len(Data)and(Data[index2][1]==numClasse))):
        index2+=1
    indexTemp = index1+int((index2-index1)*0.8)
    for j in range (index1,indexTemp):
        trainingData.append(Data[j])
    for j in range (indexTemp,index2):
        testData.append(Data[j])
    index1 = index2
    numClasse+=1 


In [5]:
random.shuffle(trainingData)

In [6]:
featuresTrain = []
labelsTrain = []
featuresTest =[]
labelsTest = []


In [7]:
for feature , label in trainingData:
    featuresTrain.append(feature)
    labelsTrain.append(label)
for feature , label in testData:
    featuresTest.append(feature)
    labelsTest.append(label)


In [8]:
featuresTrain = np.array(featuresTrain).reshape(-1,imgSize,imgSize,1)
labelsTrain = np.array(labelsTrain)

featuresTest = np.array(featuresTest).reshape(-1,imgSize,imgSize,1)
labelsTest = np.array(labelsTest)

In [9]:
hell= []
for i in range(len(labelsTrain)):
    if(labelsTrain[i]==0):
        hell.append([1,0,0,0,0,0])
    elif (labelsTrain[i]==1):
        hell.append([0,1,0,0,0,0])
    elif (labelsTrain[i]==2):
        hell.append([0,0,1,0,0,0])
    elif (labelsTrain[i]==3):
        hell.append([0,0,0,1,0,0])
    elif (labelsTrain[i]==4):
        hell.append([0,0,0,0,1,0])
    elif (labelsTrain[i]==5):
        hell.append([0,0,0,0,0,1])
    
    
labelsTrain = np.array(hell)

In [10]:
featuresTrain = tf.keras.utils.normalize(featuresTrain, axis=1)  # scales data between 0 and 1
featuresTest = tf.keras.utils.normalize(featuresTest, axis=1) 
print(featuresTrain.shape)

(4127, 64, 64, 1)


In [11]:
img_height,img_width = 64,64 
num_classes = 6

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\hala\AppData\Roaming\Python\Python37\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
modelFord = Model(inputs = base_model.input, outputs = predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
modelFord.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
modelFord.fit(featuresTrain, labelsTrain, epochs = 100, batch_size = 64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
4127/4127 [==============================] - 372s 90ms/step - loss: 3.2869 - acc: 0.1972
Epoch 2/100
4127/4127 [==============================] - 322s 78ms/step - loss: 2.7250 - acc: 0.2295
Epoch 3/100
4127/4127 [==============================] - 279s 68ms/step - loss: 2.4696 - acc: 0.2404
Epoch 4/100
4127/4127 [==============================] - 275s 67ms/step - loss: 2.2606 - acc: 0.2680
Epoch 5/100
4127/4127 [==============================] - 254s 62ms/step - loss: 2.2060 - acc: 0.2774
Epoch 6/100
4127/4127 [==============================] - 262s 63ms/step - loss: 2.0395 - acc: 0.3264
Epoch 7/100
4127/4127 [==============================] - 260s 63ms/step - loss: 1.9164 - acc: 0.3768
Epoch 8/100
4127/4127 [==============================] - 241s 58ms/step - loss: 1.6823 - acc: 0.4664
Epoch 9/100
4127/4127 [==============================] - 246s 60ms/step - loss: 1.5029 - acc: 0.5307
Epoch 10/100
4127/4127 [===================

4127/4127 [==============================] - 266s 64ms/step - loss: 0.0704 - acc: 0.9782
Epoch 79/100
4127/4127 [==============================] - 231s 56ms/step - loss: 0.1314 - acc: 0.9629
Epoch 80/100
4127/4127 [==============================] - 236s 57ms/step - loss: 0.1341 - acc: 0.9615
Epoch 81/100
4127/4127 [==============================] - 250s 60ms/step - loss: 0.1063 - acc: 0.9656
Epoch 82/100
4127/4127 [==============================] - 246s 60ms/step - loss: 0.0585 - acc: 0.9792
Epoch 83/100
4127/4127 [==============================] - 256s 62ms/step - loss: 0.0558 - acc: 0.9847
Epoch 84/100
4127/4127 [==============================] - 245s 59ms/step - loss: 0.0426 - acc: 0.9857
Epoch 85/100
4127/4127 [==============================] - 242s 59ms/step - loss: 0.0402 - acc: 0.9891
Epoch 86/100
4127/4127 [==============================] - 269s 65ms/step - loss: 0.0543 - acc: 0.9857
Epoch 87/100
4127/4127 [==============================] - 258s 63ms/step - loss: 0.0286 - acc: 

In [15]:
modelFord.save('modelFord.h5')

In [16]:
labelsTrainTransfom= []
for i in range(len(labelsTest)):
    if(labelsTest[i]==0):
        labelsTrainTransfom.append([1,0,0,0,0,0])
    elif (labelsTest[i]==1):
        labelsTrainTransfom.append([0,1,0,0,0,0])
    elif (labelsTest[i]==2):
        labelsTrainTransfom.append([0,0,1,0,0,0])
    elif (labelsTest[i]==3):
        labelsTrainTransfom.append([0,0,0,1,0,0])
    elif (labelsTest[i]==4):
        labelsTrainTransfom.append([0,0,0,0,1,0])
    elif (labelsTest[i]==5):
        labelsTrainTransfom.append([0,0,0,0,0,1])
labelsTest = np.array(labelsTrainTransfom)

In [18]:
preds = modelFord.evaluate(featuresTest,labelsTest)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1036/1036 [==============================] - 22s 21ms/step
Loss = 4.768860765405603
Test Accuracy = 0.2944015442864775
